In [1]:
import gym
import gym_watten
from gym_watten.envs.watten_env import Color, Value
import numpy as np
from time import sleep
import random
from tqdm import tqdm

In [2]:
env = gym.make("Watten-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'gym_watten.envs.watten_env.WattenEnv'>' has deprecated methods. Compatibility code invoked.


# Search

In [3]:
def search(obs):
    global sample_outputs, sample_inputs, next_index
    state = env.get_state()
    n = len(env.players[env.current_player].hand_cards)   
    p = 0
    local_index = next_index
    next_index += 1
    sample_inputs[0][local_index] = np.array(obs[0])
    sample_inputs[1][local_index] = np.array(obs[1])
    current_player = env.current_player
    
    for i in range(n):
        card_id = env.players[env.current_player].hand_cards[i].id
        obs, rew, is_done, _ = env.step(card_id)
        
        if is_done:
            sample_outputs[local_index][card_id] = (rew[0] > 0)
        else:
            sample_outputs[local_index][card_id] = ((1 - search(obs)) if current_player != env.current_player else search(obs))
        env.set_state(state)
        p += sample_outputs[local_index][card_id]
        
    return sample_outputs[local_index].max()

In [4]:
def reset_samples(iterations):
    global sample_outputs, sample_inputs, next_index, number_of_samples
    sample_outputs = [np.zeros((30000*iterations, 32), dtype=float), np.zeros((30000*iterations, 1), dtype=int)]
    sample_inputs = [np.zeros((30000*iterations, 4, 8, 2), dtype=int), np.zeros((30000*iterations, 4), dtype=int)]
    next_index = 0
    number_of_samples = 0
    
def postprocess_samples():
    global sample_outputs, sample_inputs,next_index
    for i in range(len(sample_inputs)):
        sample_inputs[i] = sample_inputs[i][:next_index]
    for i in range(len(sample_outputs)):
        sample_outputs[i] = sample_outputs[i][:next_index]
    
    #exp = np.exp(sample_outputs)
    #exp[np.where(sample_outputs == -1)] = 0
    #sample_outputs = exp / np.expand_dims(np.sum(exp, axis=-1), -1)
    
def generate_training_data(iterations=1):
    global sample_outputs, sample_inputs,next_index
    reset_samples(iterations)
    
    for i in range(iterations):
        obs = env.reset()
        search(obs)    
        
    postprocess_samples()  
    return sample_inputs, sample_outputs

# MCTS

In [87]:
class State:
    def __init__(self, p, env_state, current_player=0, end_v=0):
        self.childs = []
        self.n = 0
        self.w = 0
        self.p = p
        self.env_state = env_state
        self.end_v = end_v
        self.current_player = current_player
        
    def is_leaf_node(self):
        return len(self.childs) == 0
    
    def size(self):
        size = 1
        for child in self.childs:
            size += child.size()
        return size
    
def mcts_sample(state):
    
    if state.is_leaf_node():
        if state.end_v != 0:
            v = state.end_v
        else:
            env.set_state(state.env_state)

            p, v = model.predict_single(env.regenerate_obs())

            hand_cards = env.players[env.current_player].hand_cards[:]        
            current_player = env.current_player
            for card in hand_cards:
                obs, rew, is_done, _ = env.step(card.id)

                new_state = State(p[card.id], env.get_state(), current_player, 0 if not is_done else (1 if rew[0] > 0 else -1))
                state.childs.append(new_state)

                env.set_state(state.env_state)

            v = v[0]  
    else:
        n_sum = 0
        for child in state.childs:
            n_sum += child.n
        
        max_u = 0
        max_child = None
        for child in state.childs:
            if child.end_v != 0:
                u = child.end_v
            elif child.n > 0:
                u = child.w / child.n 
            else:
                u = 0
            
            u += child.p * np.sqrt(n_sum) / (1 + child.n)

            if max_child is None or u > max_u:
                max_u, max_child = u, child

        v = mcts_sample(max_child) * (-1 if max_child.current_player != state.current_player else 1)
        
    state.w += v 
    state.n += 1
    return v

def mcts_game_step(root):
    for i in range(50):
        mcts_sample(root)
        
    p = [child.n for child in root.childs]
    p /= np.sum(p)
    return np.random.choice(np.arange(0, len(p)), p=p), p
    
def mcts_game():
    global sample_inputs, sample_outputs, next_index, number_of_samples
    obs = env.reset()
    root = State(1, env.get_state())
    is_done = False
    start_index = next_index
    
    while not is_done:
        sample_inputs[0][next_index] = np.array(obs[0])
        sample_inputs[1][next_index] = np.array(obs[1])
        sample_outputs[1][next_index] = 1 if env.current_player is 0 else -1
        
        game_state = env.get_state()
        a, p = mcts_game_step(root)
        env.set_state(game_state)
        
        for i, card in enumerate(env.players[env.current_player].hand_cards):
            sample_outputs[0][next_index][card.id] = p[i]    
        
        next_index += 1
        number_of_samples = max(number_of_samples, next_index)
        
        last_player = env.current_player
        obs, rew, is_done, _ = env.step(env.players[env.current_player].hand_cards[a].id)
        root = root.childs[a]
                
    for i in range(start_index, next_index):
        sample_outputs[1][i][0] *= (1 if last_player is 0 else -1) * (1 if rew[0] > 0 else -1)
        
def mcts_generate():
    
    for i in tqdm(range(30)):
        mcts_game()
    #postprocess_samples()

def draw_tree(root):
    print("Root: " + str(root.end_v if root.n is 0 else root.w / root.n) + " / " + str(root.n))
    for child in root.childs:
        print("Root: " + str(child.end_v if child.n is 0 else child.w / child.n) + " / " + str(child.n))

def draw_path(root):
    state = root
    while True:     
        print(str(state.current_player) + ": " + str(state.end_v if state.n is 0 else state.w / state.n) + " / " + str(state.n))
        
        if state.is_leaf_node():
            break
    
        max_u = 0
        max_child = None
        for child in state.childs:
            if child.end_v != 0:
                u = child.end_v
            elif child.n > 0:
                u = child.w / child.n 
            else:
                u = 0
                
            if max_child is None or u > max_u:
                max_u, max_child = u, child
        
        state = max_child


In [ ]:
obs = env.reset()
game_state = env.get_state()

In [ ]:
env.set_state(game_state)
root = State(1, env.get_state())
a, p = mcts_game_step(root)
print(p)

# Play

In [99]:
def match(agents, render=False):
    obs, is_done = env.reset(), False    
    
    while not is_done:
        prediction = agents[env.current_player].predict_single(obs)[0]
        
        valid_moves = np.array([1 if card in env.players[env.current_player].hand_cards else 0 for card in env.cards])       
        prediction_valid = np.ma.masked_where(valid_moves == 0, prediction)
        
        obs, rew, is_done, _ = env.step(np.argmax(prediction_valid))
        
        if render:
            env.render('human')
            sleep(1)
        
    return env.current_player

def compare(agent1, agent2):
    agents = [agent1, agent2]
    first_player_wins = 0
    
    for i in range(1000):
        start_player = random.randint(0, 1)
        winner = match([agents[start_player], agents[1 - start_player]])
        first_player_wins += ((winner == 0) == (start_player == 0))
    
    return first_player_wins / 1000      

In [100]:
compare(model, model)

0.498

# Train

In [101]:
from keras.models import Sequential, clone_model
from keras.layers import Dense, Activation, Input, LeakyReLU
from keras.layers import Conv2D, MaxPooling2D, Merge, Flatten, BatchNormalization, add
from keras.layers.merge import concatenate
from keras.models import Model

import keras.backend as K
from IPython.display import clear_output
from keras import optimizers
import math

In [102]:
class ImprovedModel(Model):
    def __init__(self, inputs, outputs):
        super().__init__(inputs, outputs)

    def predict_single(self, inputs):
        inputs = [np.expand_dims(input, 0) for input in inputs]
        outputs = model.predict(inputs)
        return [output[0] for output in outputs]

def mean_pred(y_true, y_pred):
    return K.mean(K.equal(K.gather(y_true, K.argmax(y_true, axis=-1)), K.gather(y_true, K.argmax(y_pred, axis=-1))))


def residual_layer(input_block, filters, kernel_size):

    x = conv_layer(input_block, filters, kernel_size)

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)

    x = add([input_block, x])

    x = LeakyReLU()(x)

    return (x)

def conv_layer(x, filters, kernel_size):

    x = Conv2D(
    filters = filters
    , kernel_size = kernel_size
    , padding = 'same'
    , use_bias=False
    , activation='linear'
    )(x)

    #x = BatchNormalization(axis=1)(x)
    x = LeakyReLU()(x)

    return (x)

def build_model():
    input_1 = Input((4,8,2))
    convnet = input_1

    convnet = conv_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = residual_layer(convnet, 32, (3, 3))
    convnet = conv_layer(convnet, 2, (1, 1))
    #convnet = Conv2D(32, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    #convnet = Conv2D(64, (3, 3), activation='relu', padding='same')(convnet)
    convnet = Flatten()(convnet)

    input_2 = Input((4,))

    policy_out = concatenate([convnet, input_2])
    #dense_out = Dense(128, activation='relu')(dense_out)
    #dense_out = Dense(256, activation='relu')(dense_out)
    #policy_out = Dense(256, activation='relu')(dense_out)
    policy_out = Dense(32, activation='sigmoid')(policy_out)

    value_out = concatenate([convnet, input_2])
    #dense_out = Dense(128, activation='relu')(dense_out)
    #dense_out = Dense(256, activation='relu')(dense_out)
    value_out = Dense(256, activation='relu')(value_out)
    value_out = Dense(1, activation='tanh')(value_out)

    model = ImprovedModel(inputs=[input_1, input_2], outputs=[policy_out, value_out])

    adam = optimizers.Adam()
    model.compile(optimizer=adam,
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    return model

In [103]:
model = build_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 4, 8, 2)      0                                            
__________________________________________________________________________________________________
conv2d_25 (Conv2D)              (None, 4, 8, 32)     576         input_7[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_25 (LeakyReLU)      (None, 4, 8, 32)     0           conv2d_25[0][0]                  
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 4, 8, 32)     9216        leaky_re_lu_25[0][0]             
__________________________________________________________________________________________________
leaky_re_l

In [104]:
 reset_samples(1)

In [106]:
prev_states = []
for i in range(10):
    prev_states.append(build_model())
    prev_states[-1].set_weights(model.get_weights())
    #enerate_training_data(100)    
    mcts_generate()
    clear_output()
    p = np.random.permutation(int(number_of_samples))
    train_outputs = []
    train_inputs = []
    for i in range(len(sample_outputs)):
        train_outputs.append(sample_outputs[i][p])
    for i in range(len(sample_inputs)):
        train_inputs.append(sample_inputs[i][p])
        
    # Train the model, iterating on the data in batches of 32 samples
    model.fit(train_inputs, train_outputs, epochs=1, batch_size=64)    
    
    print("New model wins " + str(compare(model, prev_states[-1]) * 100) + "%!")

Epoch 1/1
5062/5062 [==============================] - 3s 518us/step - loss: 0.4542 - dense_10_loss: 0.0179 - dense_12_loss: 0.4363 - dense_10_acc: 0.2647 - dense_12_acc: 0.6855
New model wins 50.3%!


In [107]:
compare(model, build_model())

0.512

In [ ]:
model.save('model.h5')

In [ ]:
model.load_weights('model.h5')

# Eval

In [ ]:
generate_training_data(100)    
p = np.random.permutation(len(sample_inputs))
sample_inputs, sample_outputs = sample_inputs[p], sample_outputs[p]

In [ ]:
model.evaluate(sample_inputs, sample_outputs)

# Predict

In [10]:
def generate_input(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    obs = [np.zeros([4, 8, 2]), np.zeros([4])]
    
    for card in hand_cards:
        obs[0][card.color.value][card.value.value][0] = 1
        
    if table_card is not None:
        obs[0][table_card.color.value][table_card.value.value][1] = 1

    obs[1][0] = (own_tricks == 1 or own_tricks == 3)
    obs[1][1] = (own_tricks == 2 or own_tricks == 3)

    obs[1][2] = (opponent_tricks == 1 or opponent_tricks == 3)
    obs[1][3] = (opponent_tricks == 2 or opponent_tricks == 3)
    
    return obs

In [11]:
def card(color, value):
    for card in env.cards:
        if card.value == value and card.color == color:
            return card
        
    return None

In [76]:
def calc_correct_output_sample(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    global sample_outputs
    env.reset()
    env.cards_left = env.cards[:]
    random.shuffle(env.cards_left)
    
    for card in hand_cards:
        env.cards_left.remove(card)
    env.players[0].hand_cards = hand_cards[:]
        
    env.players[1].hand_cards = []
    for i in range(len(hand_cards) - (1 if table_card is not None else 0)):
        env.players[1].hand_cards.append(env.cards_left.pop())       
    
    env.players[0].tricks = own_tricks
    env.players[1].tricks = opponent_tricks
    env.table_card = table_card
    
    #obs, rew, is_done, _ = env.step(hand_cards[0].id)
   # print(rew, is_done)

    env.render('human')
    root = State(1, env.get_state())
    a, p = mcts_game_step(root)
    draw_tree(root.childs[0])
    
    draw_tree(root.childs[1])
    
    draw_path(root)
    #reset_samples(1)
    #search(env.regenerate_obs())
    #postprocess_samples()
        
    return p#sample_outputs[0]

In [ ]:
def calc_correct_output(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):    
    correct_output = None
    for i in range(100):
        sample_outputs = calc_correct_output_sample(hand_cards, table_card, own_tricks, opponent_tricks)
        if correct_output is None:
            correct_output = sample_outputs
        else:
            correct_output += sample_outputs
    return correct_output / 100

In [ ]:
def predict(hand_cards, table_card=None, own_tricks=0, opponent_tricks=0):
    input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
    output = model.predict_single(input)[0]
    correct_output = output#calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
    print(output)
    for card in hand_cards:
        print(str(card.color) + " " + str(card.value) + " => " + str(output[card.id]) + " (" + str(correct_output[card.id]) + ")")

In [ ]:
def eval():
    correct = 0
    for i in range(100):
        own_tricks = random.randint(0, 2)
        opponent_tricks = random.randint(0, 2)
        table_card_enabled = random.randint(0, 1)
        hand_cards = random.sample(env.cards, 5 - own_tricks - opponent_tricks + table_card_enabled)
        table_card = hand_cards.pop() if table_card_enabled else None

        input = generate_input(hand_cards, table_card, own_tricks, opponent_tricks)
        output = model.predict_single(input)[0]
        correct_output = calc_correct_output(hand_cards, table_card, own_tricks, opponent_tricks)
        
        #print(output, np.argmax(output)) 
        #print(correct_output, np.argmax(correct_output))
        correct += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
            print(correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])
    return correct / 100

In [ ]:
eval()

In [ ]:
calc_correct_output_sample([
    card(Color.EICHEL, Value.ACHT), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ], None, 0, 2)


In [93]:
calc_correct_output_sample([
    card(Color.EICHEL, Value.SAU), 
    card(Color.EICHEL, Value.KOENIG),
    card(Color.HERZ, Value.SAU)
    ],card(Color.EICHEL, Value.ZEHN), 1, 0)

Root: -0.026344962418079376 / 3
Root: -0.05577976256608963 / 1
Root: -0.039243414998054504 / 1
Root: 0.9020694872732599 / 43
Root: -0.08350489661097527 / 2
Root: 0.9740189146250486 / 40
0: 0.7729556537698954 / 50
0: 0.9020694872732599 / 43
0: 0.9740189146250486 / 40
1: -1.0 / 20


array([0.06122449, 0.87755102, 0.06122449])

In [60]:
sample_outputs[1][0]

array([1])